In [133]:
import pymrio
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
import os

In [137]:
def load(year):
    path = f"C:/Master 1/Stage/EXIOBASE3/IOT_{year}_ixi.zip"
    exio3 = pymrio.parse_exiobase3(path=path)

    diag_GHG = exio3.impacts.diag_stressor('Land use Crop, Forest, Pasture')

    exio3.GHG_source = diag_GHG

    exio3.calc_all()

    df = exio3.GHG_source.D_cba.loc[(slice(None), 'Cattle farming'), :].groupby(level='region', axis=1).sum()

    new_index = [x[0] if isinstance(x, tuple) else x for x in df.index]

    df.index = new_index

    UE = ['AT', 'BE', 'BG', 'CZ', 'DE', 'DK', 'EE', 'ES', 'FI', 'FR', 'GR', 'HR', 'HU', 'IE', 'IT', 
        'LT', 'LU', 'LV', 'MT', 'NL', 'NO', 'PL', 'PT', 'RO', 'SE', 'SI', 'SK']
    MO = ['WM', 'CY']
    Europe = ['WE', 'CH', 'TR', 'GB']
    Asie_Pacifique = ['WA', 'TW', 'KR']
    Afrique = ['WF', 'ZA']

    #Création de nouvelles colonnes pour chaque nouvelle région
    df['UE'] = df[UE].sum(axis=1)
    df['Moyen-Orient'] = df[MO].sum(axis=1)
    df['Europe'] = df[Europe].sum(axis=1)
    df['Asie Pacifique'] = df[Asie_Pacifique].sum(axis=1)
    df['Afrique'] = df[Afrique].sum(axis=1)

    #Création de nouvelles lignes pour chaque nouvelle région
    df.loc['UE'] = df.loc[UE].sum()
    df.loc['Moyen-Orient'] = df.loc[MO].sum()
    df.loc['Europe'] = df.loc[Europe].sum()
    df.loc['Asie Pacifique'] = df.loc[Asie_Pacifique].sum()
    df.loc['Afrique'] = df.loc[Afrique].sum()

    #Supprime les lignes et colonnes des pays individuels inclus dans les nouvelles régions créées.
    df = df.drop(UE + MO + Europe + Asie_Pacifique + Afrique, axis=0)
    df = df.drop(UE + MO + Europe + Asie_Pacifique + Afrique, axis=1)

    order = ["Afrique", "CN", "US", "Asie Pacifique", "WL", "BR", "RU", "MX", "AU","Moyen-Orient",  "UE",  "Europe", "CA", "IN", "ID", "JP"]
    df = pd.DataFrame(df, index=order)
    df = df[order]

    region = ["Afrique", "Chine","Etats-Unis", "Asie Pacifique",  "Amérique Latine et Caraïbes", "Brésil", "Russie", "Mexique", "Australie", "Moyen-Orient", "Union européenne",  "Europe", "Canada", "Inde", "Indonésie", "Japon" ]
    df.columns = region
    df.index = region

    df_imp = df.sum(axis=0)
    
    #Division de chaque éléments des colonnes par la somme obtenue précedemment et mettre en pourcentage
    df1_norm = df.div(df_imp, axis=1) * 100
        
    df1_norm

    df_exp = df.sum(axis=1)
    
    #Division de chaque éléments des colonnes par la somme obtenue précedemment et mettre en pourcentage
    df2_norm = df.div(df_exp, axis=0) * 100
        
    df2_norm

    return df2_norm


In [146]:
df2_norm = load(2022)

c:\Users\Marie\anaconda3\Lib\site-packages\pymrio\core\mriosystem.py:900: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  Y_agg = Y.groupby(level="region", axis=1, sort=False).sum()
c:\Users\Marie\anaconda3\Lib\site-packages\pymrio\core\mriosystem.py:956: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  F_Y_agg = self.F_Y.groupby(level="region", axis=1, sort=False).sum()
c:\Users\Marie\anaconda3\Lib\site-packages\pymrio\core\mriosystem.py:900: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  Y_agg = Y.groupby(level="region", axis=1, sort=False).sum()
c:\Users\Marie\anaconda3\Lib\site-packages\pymrio\core\mriosystem.py:956: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  F_Y_agg = self.F_Y.groupby(level="region", axis=1, sort=False).sum()
c:\Use

In [147]:
df2_norm.to_csv('df_CattleFarm_Exp_2022.csv')
